In [ ]:
#|default_exp datasets

# 0d Datasets
> What's on the tin! Pytorch utils to load our datasets, to make future use of them exceptionally easy.

In [ ]:
#|export
import torch

class PointcloudWithDistancesDataset(torch.utils.data.Dataset):
    def __init__(self, pointcloud, distances, batch_size = 64):
        self.pointcloud = torch.tensor(pointcloud, dtype=torch.float32)
        self.distances = torch.tensor(distances, dtype=torch.float32)
        self.batch_size = batch_size

    def __len__(self):
        return len(self.pointcloud)
    
    def __getitem__(self, idx):
        batch_idxs = torch.randperm(len(self.pointcloud))[:self.batch_size]
        batch = {}
        batch['x'] = self.pointcloud[batch_idxs]
        batch['d'] = self.distances[batch_idxs][:,batch_idxs]
        return batch

In [ ]:
#|export
def dataloader_from_pointcloud_with_distances(pointcloud, distances, batch_size = 64):
    dataset = PointcloudWithDistancesDataset(pointcloud, distances, batch_size)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=None, shuffle=True)
    return dataloader

In [ ]:
#|export
def train_and_testloader_from_pointcloud_with_distances(
    pointcloud, distances, batch_size = 64, train_test_split = 0.8
):
    X = pointcloud
    D = distances
    split_idx = int(len(X)*train_test_split)
    X_train = X[:split_idx]
    X_test = X[split_idx:]
    D_train = D[:split_idx,:split_idx]
    D_test = D[split_idx:,split_idx:]
    trainloader = dataloader_from_pointcloud_with_distances(X_train, D_train, batch_size)
    testloader = dataloader_from_pointcloud_with_distances(X_test, D_test, batch_size)
    return trainloader, testloader

# Compiling the datasets
Though defined in other notebooks, we compile the datasets here for ease of use.

In [ ]:
#|export
from autometric.n0d1_branching_datasets import *

In [ ]:
#|export
from diffusion_curvature.datasets import *
from diffusion_curvature.utils import plot_3d